## Calculating tobin's q of S&P 500 firms Using tobin's Q


### Installing some modules

In [1]:
#%pip install lxml
#import pandas as pd
#%pip install yfinance

In [2]:
import yfinance as yf
import pandas as pd

# Step 1: Get the list of S&P500 tickers from Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
tables = pd.read_html(url)
sp500_df = tables[0]
tickers = sp500_df['Symbol'].tolist()


googl = yf.Ticker("GOOGL")
googl_info = googl.info    
liab = googl_info.get('totalLiab', None)
print("Total Liabilities of GOOGL:", liab)

Total Liabilities of GOOGL: None


In [3]:
import yfinance as yf

ticker = yf.Ticker("MMM")
# Check available info keys
print(ticker.info.keys())

# For balance sheet data:
bs = ticker.balance_sheet
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
print(bs.index )  

import yfinance as yf
import pandas as pd

# Optional: Set pandas display options so that all rows and columns are visible.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

# Step 1: Get the list of S&P 500 tickers from Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
tables = pd.read_html(url)
sp500_df = tables[0]
tickers = sp500_df['Symbol'].tolist()

def collect_data(ticker):
    """
    Collects key financial data for a given ticker.
    Returns a dictionary containing market cap, total assets, and total liabilities
    (using 'Total Liabilities Net Minority Interest') from the balance sheet.
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        market_cap = info.get('marketCap', None)
        
        # Retrieve the balance sheet DataFrame
        bs = stock.balance_sheet
        if bs is None or bs.empty or market_cap is None:
            print(f"Missing data for {ticker}.")
            return None

        # Use the most recent balance sheet date
        latest_date = bs.columns[0]

        # Extract Total Assets; check if the key exists
        total_assets = bs.loc['Total Assets', latest_date] if 'Total Assets' in bs.index else None

        # Extract Total Liabilities using the available key from your output
        total_liab = bs.loc['Total Liabilities Net Minority Interest', latest_date] if 'Total Liabilities Net Minority Interest' in bs.index else None

        if total_assets is None or total_liab is None:
            print(f"Missing balance sheet data for {ticker}. Assets: {total_assets}, Liabilities: {total_liab}")
            return None

        return {
            'market_cap': market_cap,
            'total_assets': total_assets,
            'total_liab': total_liab
        }
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        return None

def calculate_tobins_q(data):
    """
    Calculates Tobin's Q using the simplified formula:
    (Market Cap + Total Liabilities) / Total Assets.
    """
    if data is None or data['total_assets'] == 0:
        return None
    return (data['market_cap'] + data['total_liab']) / data['total_assets']

# Collect data and compute Tobin's Q for each ticker
results = {}
for ticker in tickers:
    data = collect_data(ticker)
    q_value = calculate_tobins_q(data)
    if q_value is not None:
        results[ticker] = q_value

# Create a DataFrame to display the results
df_results = pd.DataFrame(list(results.items()), columns=['Ticker', "Tobin's Q"])
print(df_results)



dict_keys(['address1', 'city', 'state', 'zip', 'country', 'phone', 'website', 'industry', 'industryKey', 'industryDisp', 'sector', 'sectorKey', 'sectorDisp', 'longBusinessSummary', 'fullTimeEmployees', 'companyOfficers', 'auditRisk', 'boardRisk', 'compensationRisk', 'shareHolderRightsRisk', 'overallRisk', 'governanceEpochDate', 'compensationAsOfEpochDate', 'irWebsite', 'executiveTeam', 'maxAge', 'priceHint', 'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose', 'regularMarketOpen', 'regularMarketDayLow', 'regularMarketDayHigh', 'dividendRate', 'dividendYield', 'exDividendDate', 'payoutRatio', 'fiveYearAvgDividendYield', 'beta', 'trailingPE', 'forwardPE', 'volume', 'regularMarketVolume', 'averageVolume', 'averageVolume10days', 'averageDailyVolume10Day', 'bid', 'ask', 'bidSize', 'askSize', 'marketCap', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 'fiftyDayAverage', 'twoHundredDayAverage', 'trailingAnnualDividendRate', 'trailingAnnualDividen

In [4]:
import yfinance as yf
import pandas as pd

# Set display options so that the full table is shown
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

# Get the list of S&P500 tickers and associated data from Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
tables = pd.read_html(url)
sp500_df = tables[0]
# The Wikipedia table typically includes columns like 'Symbol', 'Security', 'Date first added', etc.

def collect_data(ticker):
    """
    Collects financial data from yfinance for the given ticker.
    Returns a dictionary containing:
      - market_cap: Market capitalization
      - founded: Year the company was founded (if available)
      - total_assets: Total assets from the most recent balance sheet
      - total_liab: Total liabilities from 'Total Liabilities Net Minority Interest'
    If key data is missing, returns None.
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        market_cap = info.get('marketCap', None)
        # Attempt to get the founding year; if not available, default to "N/A"
        founded = info.get('founded', "N/A")
        
        # Retrieve the balance sheet DataFrame
        bs = stock.balance_sheet
        if bs is None or bs.empty or market_cap is None:
            print(f"Missing data for {ticker}.")
            return None
        
        # Use the most recent balance sheet date
        latest_date = bs.columns[0]
        
        # Extract Total Assets and Total Liabilities
        total_assets = bs.loc['Total Assets', latest_date] if 'Total Assets' in bs.index else None
        total_liab = bs.loc['Total Liabilities Net Minority Interest', latest_date] if 'Total Liabilities Net Minority Interest' in bs.index else None
        
        if total_assets is None or total_liab is None:
            print(f"Missing balance sheet data for {ticker}. Assets: {total_assets}, Liabilities: {total_liab}")
            return None
        
        return {
            'ticker': ticker,
            'market_cap': market_cap,
            'total_assets': total_assets,
            'total_liab': total_liab,
            'founded': founded
        }
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        return None

def calculate_tobins_q(data):
    """
    Calculates Tobin's Q using:
      (Market Cap + Total Liabilities) / Total Assets.
    Returns None if data is missing or if total assets is zero.
    """
    if data is None or data['total_assets'] == 0:
        return None
    return (data['market_cap'] + data['total_liab']) / data['total_assets']

results_list = []

# Iterate over the rows in the Wikipedia table
for idx, row in sp500_df.iterrows():
    ticker = row['Symbol']
    full_name = row['Security']
    # 'Date first added' may be missing for some; default to "N/A" if so
    date_added = row.get('Date first added', "N/A")
    
    data = collect_data(ticker)
    if data is None:
        continue  # Skip if we couldn't retrieve data
    
    q_value = calculate_tobins_q(data)
    if q_value is None:
        continue  # Skip if calculation failed
    
    result = {
        'Ticker': ticker,
        'Full Name': full_name,
        'Founded': data['founded'],
        'Date Added': date_added,
        'Market Cap': data['market_cap'],
        'Total Liabilities': data['total_liab'],
        "Tobin's Q": q_value
    }
    results_list.append(result)

# Create and print the final DataFrame
df_results = pd.DataFrame(results_list)
print(df_results)


Missing data for BRK.B.
Missing data for BF.B.
    Ticker                               Full Name Founded Date Added     Market Cap  Total Liabilities  Tobin's Q
0      MMM                                      3M     N/A        N/A    76749987840       3.597400e+10   2.827430
1      AOS                             A. O. Smith     N/A        N/A     9764436992       1.356500e+09   3.432388
2      ABT                     Abbott Laboratories     N/A        N/A   229189107712       3.351300e+10   3.226744
3     ABBV                                  AbbVie     N/A        N/A   348559802368       1.317970e+11   3.553960
4      ACN                               Accenture     N/A        N/A   191238291456       2.676412e+10   3.897608
5     ADBE                              Adobe Inc.     N/A        N/A   162914942976       1.612500e+10   5.922592
6      AMD                  Advanced Micro Devices     N/A        N/A   161715666944       1.165800e+10   2.504459
7      AES                       

In [5]:
print(sp500_df.columns.tolist())


['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date added', 'CIK', 'Founded']


In [6]:
import yfinance as yf
import pandas as pd

# --- Set display options ---
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

# --- Step 1: Get S&P 500 data from Wikipedia ---
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
tables = pd.read_html(url)
sp500_df = tables[0]
print("Wikipedia columns:", sp500_df.columns.tolist())

# Select relevant columns:
# The table includes: ['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 
# 'Headquarters Location', 'Date added', 'CIK', 'Founded']
wiki_data = sp500_df[['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Date added', 'Founded']].copy()
wiki_data.rename(columns={
    'Symbol': 'Ticker',
    'Security': 'Full Name',
    'GICS Sector': 'Sector',
    'GICS Sub-Industry': 'Industry',
    'Date added': 'Date Added',
    'Founded': 'Founded (Wiki)'
}, inplace=True)

# --- Step 2: Define functions to collect financial data and calculate Tobin's Q ---

def collect_data(ticker):
    """
    Collects financial data for a given ticker using yfinance.
    Returns a dictionary with:
      - Market Cap (from info)
      - Founded (from yfinance info, if available)
      - Total Assets (from the most recent balance sheet)
      - Total Liabilities (using 'Total Liabilities Net Minority Interest')
    Returns None if key data is missing.
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        market_cap = info.get('marketCap', None)
        # Optionally, get the founded year from yfinance (may not be available for all)
        founded_yf = info.get('founded', None)
        
        # Retrieve balance sheet DataFrame
        bs = stock.balance_sheet
        if bs is None or bs.empty or market_cap is None:
            print(f"Missing data for {ticker}.")
            return None
        
        # Use the most recent balance sheet column
        latest_date = bs.columns[0]
        total_assets = bs.loc['Total Assets', latest_date] if 'Total Assets' in bs.index else None
        total_liab = bs.loc['Total Liabilities Net Minority Interest', latest_date] if 'Total Liabilities Net Minority Interest' in bs.index else None
        
        if total_assets is None or total_liab is None:
            print(f"Missing balance sheet items for {ticker}. Assets: {total_assets}, Liabilities: {total_liab}")
            return None
        
        return {
            'Ticker': ticker,
            'Market Cap': market_cap,
            'Founded (YF)': founded_yf,
            'Total Assets': total_assets,
            'Total Liabilities': total_liab
        }
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        return None

def calculate_tobins_q(data):
    """
    Calculates Tobin's Q using:
      (Market Cap + Total Liabilities) / Total Assets.
    Returns None if data is missing or if Total Assets is zero.
    """
    if data is None or data['Total Assets'] == 0:
        return None
    return (data['Market Cap'] + data['Total Liabilities']) / data['Total Assets']

# --- Step 3: Collect financial data and compute Tobin's Q for each ticker ---
results_list = []
for ticker in wiki_data['Ticker']:
    data = collect_data(ticker)
    if data is None:
        continue
    q_value = calculate_tobins_q(data)
    if q_value is None:
        continue
    data["Tobin's Q"] = q_value
    results_list.append(data)

financial_df = pd.DataFrame(results_list)

# --- Step 4: Merge Wikipedia data with financial data ---
merged_df = pd.merge(wiki_data, financial_df, on='Ticker', how='inner')

# Display the merged DataFrame
Tobins_q = merged_df

Tobins_q.to_csv('Tobins_q.csv', index=False)


Wikipedia columns: ['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date added', 'CIK', 'Founded']
Missing data for BRK.B.
Missing data for BF.B.


In [7]:
merged_df.sort_values(by=["Tobin's Q"])

,Ticker,Full Name,Sector,Industry,Date Added,Founded (Wiki),Market Cap,Founded (YF),Total Assets,Total Liabilities,Tobin's Q
254,IVZ,Invesco,Financials,Asset Management & Custody Banks,2008-08-21,1935,6495648768,None,2.700890e+10,1.134010e+10,0.660366
14,ARE,Alexandria Real Estate Equities,Real Estate,Office REITs,2017-03-20,1994,12904978432,None,3.752745e+10,1.512899e+10,0.747026
470,VTRS,Viatris,Health Care,Pharmaceuticals,2004-04-23,1961,10281406464,None,4.150090e+10,2.286540e+10,0.798701
360,PARA,Paramount Global,Communication Services,Movies & Entertainment,1994-09-30,2019 (Paramount Pictures 1912),8302555136,None,4.617200e+10,2.939000e+10,0.816351
13,ALB,Albemarle Corporation,Materials,Specialty Chemicals,2016-07-01,1994,7210854400,None,1.660965e+10,6.409961e+09,0.820054
276,KHC,Kraft Heinz,Consumer Staples,Packaged Foods & Meats,2015-07-06,2015 (1869),33825572864,None,8.828700e+10,3.896200e+10,0.824443
481,WBD,Warner Bros. Discovery,Communication Services,Broadcasting,2022-04-11,2022 (Warner Bros. 1923),21347508224,None,1.045600e+11,6.962200e+10,0.870022
201,BEN,Franklin Resources,Financials,Asset Management & Custody Banks,1998-04-30,1947,10508380160,None,3.246450e+10,1.789970e+10,0.875051
318,MHK,Mohawk Industries,Consumer Discretionary,Home Furnishings,2013-12-23,1878,6462862336,None,1.277860e+10,5.221700e+09,0.914385
326,MOS,Mosaic Company (The),Materials,Fertilizers & Agricultural Chemicals,2011-09-26,2004 (1865 / 1909),9678688256,None,2.292400e+10,1.130930e+10,0.915547
